In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
%matplotlib inline

In [ ]:
calendar_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv', parse_dates=['date'], usecols=['date','d'])
sample_sub = pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv', index_col='id')

# udvælger kun de datoer som ligger i sales_train_validation

calendar_stv = calendar_df[:1913]
sales = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv', index_col='id')


# Trend i data

In [ ]:
sales_mean = sales.mean()
sales_mean.index = calendar_stv['date']
sales_mean_trans = sales_mean.transpose()

In [ ]:
decomp = seasonal_decompose(sales_mean_trans['2014':], period=7)
plt.rcParams['figure.figsize'] = [20, 15]
decomp.plot()
plt.show()

In [ ]:
fig = px.scatter(sales_mean_trans, x=sales_mean_trans.index, y=sales_mean_trans.values, trendline='ols')
fig.show()

In [ ]:
diffed = sales_mean_trans.diff().dropna()
fig = px.scatter(diffed, x=diffed.index, y=diffed.values, trendline='ols')
fig.show()

Der er en opadgående trend i datasættet, som bliver fjernet af at diffe.

In [ ]:
diffed_season = sales_mean_trans.diff().diff(7).dropna()
fig = px.scatter(diffed_season, x=diffed_season.index, y=diffed_season.values, trendline='ols')
fig.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
results = adfuller(sales_mean_trans)
print('ingen dif')
print('  ADF Statistic: {}'.format(results[0]))
print('  p-value: {}'.format(results[1]))
results = adfuller(diffed)
print('diffed')
print('  ADF Statistic: {}'.format(results[0]))
print('  p-value: {}'.format(results[1]))
results = adfuller(diffed_season)
print('diffed+seasonal diffed 7 dage')
print('  ADF Statistic: {}'.format(results[0]))
print('  p-value: {}'.format(results[1]))
results = adfuller(diffed.diff(30).dropna())
print('diffed+seasonal diffed 30 dage')
print('  ADF Statistic: {}'.format(results[0]))
print('  p-value: {}'.format(results[1]))
results = adfuller(diffed.diff(365).dropna())
print('diffed+seasonal diffed 365 dage')
print('  ADF Statistic: {}'.format(results[0]))
print('  p-value: {}'.format(results[1]))

Da jeg ved at der er en sæson komponent i data, plotter jeg mine acf og pacf med det diffede sæt

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
fig,(ax1, ax2) = plt.subplots(2,1, figsize=(20,10))
plot_acf(diffed, zero=False, ax=ax1, lags=21)
plot_pacf(diffed, zero=False, ax=ax2, lags=21)
plt.show()

Diffet datasæt - generel trend er fjernet
- Ved at kigge på ACF tyder det på at en mønster gentages for hver 7 dage
- Vi prøver at fjerne det mønster:


In [ ]:
diffed_7 = diffed.diff(7).dropna()
fig,(ax1, ax2) = plt.subplots(2,1, figsize=(20,10))
plot_acf(diffed_7, zero=False, ax=ax1, lags=21)
plot_pacf(diffed_7, zero=False, ax=ax2, lags=21)
plt.show()

på nuværende tidspunkt er datasættet blevet diffet en gang og en gang med en sæson på 7.

SARIMAX består af parametrene (p,d,q)(P,D,Q)s
- d og D og s værdierne kender vi:
  - d=1 da datasættet skal diffes for at fjerne trenden
  - D=1 fordi vi har obseveret seasonality
  - s=7 fordi vi har sat seasonality til en uge
- På acf-grafen kan man umiddelbart aflæse antallet af lags der skal bruges i MA(q)
  - q=2
- på pacf-grafen kan man umiddelbart aflæse antallet af lags på AR(p)
  - p=4
- Så mangler vi P og Q værdierne:

Nu plotter vi acf og pacf med lags der svarer til intevaller på 7 dage

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
fig,(ax1, ax2) = plt.subplots(2,1, figsize=(20,10))
normal_lags = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
lags7 =  [element * 7 for element in normal_lags]

plot_acf(diffed_7, lags=lags7, ax=ax1)
plot_pacf(diffed_7, lags=lags7, ax=ax2)
plt.show()

De samme principper gælder som ved p,q værdierne: P aflæses på pacf og Q aflæses på acf
- Q = 1, lag 0 sæller ikke med som en faktor
- P = 5,6 måske 9 
  - Grafen er mere tvetydig for aflæsning

In [ ]:
%%time
model = SARIMAX(sales_mean_trans, order=(4,1,2), seasonal_order=(5,1,1,7))
results = model.fit()
print(results.aic, results.bic)

In [ ]:
print(results.summary())

In [ ]:
forecast = results.get_prediction(start=-28)
mean_forecast = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

#confidence_intervals
lower_limits = confidence_intervals.loc[:,'lower y']
upper_limits = confidence_intervals.loc[:,'upper y']
plt.figure(figsize=(20,10))
#Plot prediction
plt.plot(sales_mean_trans[-35:])
plt.plot(mean_forecast.index,
         mean_forecast.values,
         color='red',
         label='forecast')
#shade uncertainty area
plt.fill_between(mean_forecast.index, lower_limits, upper_limits, color='pink')
plt.show()

In [ ]:
#mean_forecast
RMSE = np.mean((sales_mean_trans[-28:] - mean_forecast)**2)**0.5
print("Sarimax(4,1,2)(5,1,1)7\n - RMSE score: {}\n - fitting Tid: {}".format(RMSE, '110 sekunder'))
